# Rulebased and Random Backchanneling

### Python Imports

In [ ]:
import numpy as num
import math
import time 
import random
from random import randint
from websocket import create_connection

import aubio
import pyaudio
import wave

### Create Socket Connection

In [ ]:
ws = create_connection("ws://url:port")

### Start Greeter Message
“Hi! I understood that you witnessed a crime or an attempted crime. Could you please tell me as much as possible of what you saw from the offenders, victims and other witnesses? Please tell every detail you remember from the moment the video started, even if they seem trivial to you.”

In [ ]:
ws.send(";GREETER")

### Start Ending Message
“Thank you for the information! I think i have a clear view of what happened. Have a nice day.”

In [ ]:
ws.send(";ENDING")

### Produce Backchannel
The type of backchannel is determined using the backchannel frequency according to S. White, “Backchannels across cultures: A study of americans and japanese,” Language in Society, vol. 18, no. 1, p. 5976, 1989.

In [ ]:
def produceBackchannel():
    choice = num.random.choice(5, 1, p=[0.43, 0.19, 0.18, 0.14, 0.06])[0]

    if choice == 0:
        print "mmhm!"
        return "MHMMM"
    elif choice == 1:
        print "yeah!"
        return "YEAH"
    elif choice == 2:
        print "uh-huh!"
        return "UHUH"
    elif choice == 3:
        print "oh!"
        return "OHH"
    elif choice == 4:
        print "hmm!"
        return "HMMMM"

## Backchanneling Methods

### Random Backchanneling Method

In [ ]:
items_per_minute = 10

def newRandom(items_per_minute):
    rand_items = math.ceil(60 / items_per_minute)
    return random.randint(0,rand_items)

while True:
    rand_val = newRandom(items_per_minute)
    
    time.sleep((60 / items_per_minute) - rand_val)
    
    backchannel = produceBackchannel()
    ws.send(";" + backchannel)
    
    time.sleep(rand_val)

### Rulebased Backchanneling
K. P. Truong, R. Poppe, and D. Heylen, “A rule-based backchannel prediction model using pitch and pause information,” Ph.D. dissertation, Electrical Engi- neering, Mathematics and Computer Science (EEMCS), Sep. 2010.

In [ ]:
silence_threshold = -70.

p = pyaudio.PyAudio()

frames = 1024 
sample_rate = 44100

silence_threshold = -40

pitch_length_threshold = .100
speech_length_threshold = .700
previous_backchannel_threshold = 1.400
pitch_threshold = 30

wait_length = .700

total_frames = 0

length_silence = 0
length_speech = 0
length_speech_old = 0
previous_backchannel = 0

pitch_array = []

# Open stream.
stream = p.open(format=pyaudio.paFloat32,
    channels=1, rate=sample_rate, input=True,
    frames_per_buffer=frames)

# Aubio's pitch detection.
pDetection = aubio.pitch("default", 2048,
    2048//2, sample_rate)
# Set unit.
pDetection.set_unit("Hz")
pDetection.set_silence(silence_threshold)

while True:
    data = stream.read(1024)
    samples = num.fromstring(data, dtype=aubio.float_type)
    
    total_frames += len(samples)
    
    is_silence = aubio.silence_detection(samples, silence_threshold);
    
    if(is_silence):
        length_silence += len(samples)
        length_speech_old = length_speech
        length_speech = 0
    else:
        length_silence = 0
        length_speech_old = 0
        length_speech += len(samples)
        
    pitch = pDetection(samples)[0] 
    pitch_array.append(pitch)
    
    previous_backchannel += len(samples)
        
    last_pitches = int(math.ceil((sample_rate * pitch_length_threshold) / len(samples)))
    
    sel_pitches =  pitch_array[(-1 * last_pitches):]
        
    if(
        length_silence >= (sample_rate * silence_threshold) and
        length_speech_old >= (sample_rate * speech_length_threshold) and
        previous_backchannel >= (sample_rate * previous_backchannel_threshold) and
        abs(sel_pitches[0] - sel_pitches[len(sel_pitches) - 1]) >= pitch_threshold
    ):
        
        time.sleep(.400)
        
        backchannel = produceBackchannel()
        ws.send(";" + backchannel)
        
        previous_backchannel = 0

In [ ]:
ws.close()